In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import datetime

In [23]:
VANTAA_Bikers=pd.read_csv('VantaaBikers_cleanDataset.csv')
VANTAA_Bikers.Date=pd.to_datetime(VANTAA_Bikers.Date,format='%Y-%m-%d')
VANTAA_Bikers=VANTAA_Bikers.set_index(['Date','Time'])
cindex_VANTAA=VANTAA_Bikers.columns[1:]
#VANTAA_Bikers.info()

ESPOO_Bikers=pd.read_csv('EspooBikers_cleanDataset.csv')
ESPOO_Bikers.Date=pd.to_datetime(ESPOO_Bikers.Date,format='%Y-%m-%d')
ESPOO_Bikers=ESPOO_Bikers.set_index(['Date','Time'])
cindex_ESPOO=ESPOO_Bikers.columns[1:]
#ESPOO_Bikers.info()

HELSINKI_Bikers=pd.read_csv('HelsinkiBikers_cleanDataset.csv')
HELSINKI_Bikers.Date=pd.to_datetime(HELSINKI_Bikers.Date,format='%Y-%m-%d')
HELSINKI_Bikers=HELSINKI_Bikers.set_index(['Date','Time'])
HELSINKI_Bikers.Baana=HELSINKI_Bikers.Baana*1.0
cindex_HELSINKI=HELSINKI_Bikers.columns[1:]

#HELSINKI_Bikers.info()

ALL_Bikers=HELSINKI_Bikers.copy()
ALL_Bikers=ALL_Bikers.join(VANTAA_Bikers.iloc[:,1:],how='left')
ALL_Bikers=ALL_Bikers.join(ESPOO_Bikers.iloc[:,1:],how='left')
ALL_Bikers.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51216 entries, (Timestamp('2014-01-01 00:00:00'), '00:00:00') to (Timestamp('2019-10-31 00:00:00'), '23:00:00')
Data columns (total 85 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Weekday                                        51216 non-null  object 
 1   Auroransilta                                   17330 non-null  float64
 2   Etelaesplanadi                                 50546 non-null  float64
 3   Huopalahti (station)                           39431 non-null  float64
 4   Kaisaniemi/Elaintarhanlahti                    51214 non-null  float64
 5   Kaivokatu                                      17328 non-null  float64
 6   Kulosaari bridge south                         39431 non-null  float64
 7   Kulosaaren silta po.                           39431 non-null  float64
 8   Kuusisaarentie                          

In [24]:
pyhapaivat=pd.DataFrame();
for vuosi in range(2014,2020):
    table=pd.read_html(f'https://www.webcal.fi/fi-FI/pyhat.php?y={vuosi}')
    table=table[0].iloc[:,1:4]
    table.iloc[:,2]=pd.to_datetime(table.iloc[:,2], format='%d.%m.%Y', errors='coerce')
    table.columns=['Pyhapaiva','Viikonpaiva',str(vuosi)]
    table=table.drop('Viikonpaiva',axis=1).set_index('Pyhapaiva')    
    if vuosi==2014:
        pyhapaivat=table
    else:
        pyhapaivat=pyhapaivat.join(table)

pyhapaivat_list=[date for date in pd.DatetimeIndex(pyhapaivat.iloc[:,0]).date]
for n in range(1,6):
    pyhapaivat_list.extend([date for date in pd.DatetimeIndex(pyhapaivat.iloc[:,n]).date])
pyhapaivat_set=set(pyhapaivat_list)

def onko_pyhapaiva(pdf):
    return (np.array([date in pyhapaivat_set for date in pd.DatetimeIndex(pdf.index).date]))




ALL_Bikers['pyhapaiva']=onko_pyhapaiva(ALL_Bikers.reset_index().set_index('Date'))
ALL_Bikers['nData']=ALL_Bikers.iloc[:,1:-1].T.count().astype('Int64')
ALL_Bikers['nData_VAN']=ALL_Bikers.loc[:,cindex_VANTAA].T.count().astype('Int64')
ALL_Bikers['nData_ESP']=ALL_Bikers.loc[:,cindex_ESPOO].T.count().astype('Int64')
ALL_Bikers['nData_HEL']=ALL_Bikers.loc[:,cindex_HELSINKI].T.count().astype('Int64')


#ALL_Bikers


columns=ALL_Bikers.columns
columns=columns[0:1].append(columns[-5:]).append(columns[1:-5])
columns
ALL_Bikers=ALL_Bikers.reindex(columns=columns)

ALL_Bikers.to_csv('All_bikers.csv')
ALL_Bikers

Weekday  pyhapaiva  nData  nData_VAN  nData_ESP  \
Date       Time                                                       
2014-01-01 00:00:00     Wed       True      7          0          0   
           01:00:00     Wed       True      7          0          0   
           02:00:00     Wed       True      7          0          0   
           03:00:00     Wed       True      7          0          0   
           04:00:00     Wed       True      7          0          0   
...                     ...        ...    ...        ...        ...   
2019-10-31 19:00:00     Thu      False     19          0          0   
           20:00:00     Thu      False     19          0          0   
           21:00:00     Thu      False     19          0          0   
           22:00:00     Thu      False     19          0          0   
           23:00:00     Thu      False     19          0          0   

                     nData_HEL  Auroransilta  Etelaesplanadi  \
Date       Time                                                
2014-01-01 00:00:00          7           NaN             7.0   
           01:00:00          7           NaN             5.0   
           02:00:00          7           NaN             2.0   
           03:00:00          7           NaN             5.0   
           04:00:00          7           NaN             1.0   
...                        ...           ...             ...   
2019-10-31 19:00:00         19          24.0            60.0   
           20:00:00         19          32.0            51.0   
           21:00:00         19          21.0            23.0   
           22:00:00         19           8.0            13.0   
           23:00:00         19           3.0            16.0   

                     Huopalahti (station)  Kaisaniemi/Elaintarhanlahti  ...  \
Date       Time                                                         ...   
2014-01-01 00:00:00                   NaN                          1.0  ...   
           01:00:00                   NaN                          3.0  ...   
           02:00:00                   NaN                          3.0  ...   
           03:00:00                   NaN                          2.0  ...   
           04:00:00                   NaN                          4.0  ...   
...                                   ...                          ...  ...   
2019-10-31 19:00:00                  47.0                         46.0  ...   
           20:00:00                  34.0                         39.0  ...   
           21:00:00                  19.0                         13.0  ...   
           22:00:00                   8.0                          9.0  ...   
           23:00:00                   5.0                          8.0  ...   

                     Rantaradanraitti, Helsingin raja (Viacount 2)  \
Date       Time                                                      
2014-01-01 00:00:00                                            NaN   
           01:00:00                                            NaN   
           02:00:00                                            NaN   
           03:00:00                                            NaN   
           04:00:00                                            NaN   
...                                                            ...   
2019-10-31 19:00:00                                            NaN   
           20:00:00                                            NaN   
           21:00:00                                            NaN   
           22:00:00                                            NaN   
           23:00:00                                            NaN   

                     Soukka (Viacount)  Suomenlahdentie (Eco-Counter)  \
Date       Time                                                         
2014-01-01 00:00:00                NaN                            NaN   
           01:00:00                NaN                            NaN   
           02:00:00             